In [1]:
import pickle as pkl
import pandas as pd
import numpy as np
import pytorch_nndct
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from pytorch_nndct import Inspector
batch_size = 1
# load data
data = pd.read_pickle("RML2016.10a_dict.pkl", compression='infer')
qpsk_2_data_all = data[('QPSK', 2)]
bpsk_2_data_all = data[('BPSK', 2)]

# labels
qpsk_labels = [0] * 1000  # QPSK = 0
bpsk_labels = [1] * 1000  # BPSK = 1

# combine the data lables
data_combined = np.concatenate((qpsk_2_data_all, bpsk_2_data_all), axis=0)
labels_combined = qpsk_labels + bpsk_labels

# convert labels to NumPy array and then to PyTorch tensor with Long data type
labels_combined = np.array(labels_combined, dtype=np.int64)
labels_combined = torch.from_numpy(labels_combined).long()

# convert 2 PyTorch tensor
data_combined = torch.from_numpy(data_combined).float()

# convert labels 2 NumPy array and then 2 PyTorch tensor
labels_combined = np.array(labels_combined)
labels_combined = torch.from_numpy(labels_combined)

# break into training + testing
test_size = 0.2  # Adjust the test size as needed
data_train, data_test, labels_train, labels_test = train_test_split(
    data_combined, labels_combined, test_size=test_size, random_state=42)

class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # access a single data sample and label
        sample = self.data[idx]
        label = self.labels[idx]
    
        # Convert sample, min_vals, and max_vals to PyTorch tensors
        sample = torch.tensor(sample, dtype=torch.float32)
        min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
        max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
        
        #normalize
        epsilon = 1e-10
        normalized_sample = 2 * (sample - min_vals.unsqueeze(1)) / (max_vals.unsqueeze(1) - min_vals.unsqueeze(1) + epsilon) - 1
    
        return normalized_sample, label

train_dataset = MyDataset(data_train, labels_train)
test_dataset = MyDataset(data_test, labels_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


[VAIQ_NOTE]: Loading NNDCT kernels...


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [2]:
import torch
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self, num_classes):
        super(CNN1D, self).__init__()
        
        in_channels = 2
        in_features = 128
        
        # define convolutional layers
        self.conv1 = nn.Conv1d(in_channels, 64, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool1d(kernel_size=2)
        
        # efine the fully connected layers
        self.fc1 = nn.Linear(128 * (in_features // 4), 256)  # in_features // 4 due to max pooling
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        # input shape: (batch_size, 2, 128)
        
        # convolutional layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        
        # flatten (before fully connected layers)
        x = x.view(x.size(0), -1)
        
        # fully connected layers
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        
        return x

# initialize model
num_classes = 2 #BPSK / QPSK
model = CNN1D(num_classes)

print(model) #model info

CNN1D(
  (conv1): Conv1d(2, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu1): ReLU()
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (relu2): ReLU()
  (maxpool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4096, out_features=256, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)


In [9]:
# Specify a target name or fingerprint you want to deploy on
target = "DPUCAHX8L_ISA0_SP"
#DPUCVDX8G
# Initialize inspector with target
inspector = Inspector(target)
# Note: visualization of inspection results relies on the dot engine.If you don't install dot successfully, set 'image_format = None' when inspecting.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN1D(2)

#dummy_data, labels = train_loader
data_dummy = []
for data, labels in train_loader:
    data_dummy = data
    break

#dummy_input = torch.randn(1, 128, 3, 3)
inspector.inspect(model, (data_dummy,), device=device, output_dir="inspect", image_format="png") 


[VAIQ_NOTE]: Inspector is on.

[VAIQ_NOTE]: =>Start to inspect model...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing CNN1D...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_CNN1D_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


/tmp/ipykernel_118/822561179.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_118/822561179.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_118/822561179.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)
██████████████████████████████████████████████████| 13/13 [00:00<00:00, 2666.83it/s, OpInfo: name = ret


[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(inspect/CNN1D.py)

[VAIQ_WARN]: CNN1D::308 is not tensor.

[VAIQ_NOTE]: Find subgraph for convlike_fix_18:
node name:CNN1D::CNN1D/Linear[fc1]/ret.19, op type:nndct_dense, output shape: [1, 256]
node name:CNN1D::CNN1D/ReLU[relu3]/ret.21, op type:nndct_relu, output shape: [1, 256]



[VAIQ_NOTE]: Find subgraph for reshape_fix_1:
node name:CNN1D::CNN1D/MaxPool1d[maxpool2]/ret.13_sink_transpose_0, op type:nndct_permute, output shape: [1, 128, 32]



[VAIQ_NOTE]: Find subgraph for reshape_fix_1:
node name:CNN1D::CNN1D/ret.17, op type:nndct_reshape, output shape: [1, 4096]



[VAIQ_NOTE]: Find subgraph for convlike_fix_20:
node name:CNN1D::CNN1D/Linear[fc2]/ret, op type:nndct_dense, output shape: [1, 2]





I20240215 10:18:54.841639   118 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240215 10:18:54.841661   118 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240215 10:18:54.841665   118 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCAHX8L_ISA0_SP
I20240215 10:18:54.841737   118 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_dense_nndct_relu_gbpUSIH6h2JmT3Li, with op num: 9
I20240215 10:18:54.841740   118 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
I20240215 10:18:54.874176   118 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 1
I20240215 10:18:54.874205   118 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.
I20240215 10:18:54.878021   118 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20240215 10:18:54.878032   118 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20240215 10:18:54.878036   118 compile_p

KeyError: (0, 2, 1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader


learning_rate = 0.001
num_epochs = 10

# loss function and optimizer definition
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# training
for epoch in range(num_epochs):
    model.train()  # Set the model in training mode
    
    for data, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(data)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
    
    # print loss
    print(f'Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}')

# evaluate the model on test data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        # predicted and actual labels
        for i in range(len(labels)):
            print(f'Predicted: {predicted[i]}, Actual: {labels[i]}')

accuracy = 100 * correct / total
print(f'Test Accuracy: {accuracy:.2f}%')

/tmp/ipykernel_118/822561179.py:53: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(sample, dtype=torch.float32)
/tmp/ipykernel_118/822561179.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  min_vals = torch.tensor(sample.min(axis=1).values, dtype=torch.float32)
/tmp/ipykernel_118/822561179.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  max_vals = torch.tensor(sample.max(axis=1).values, dtype=torch.float32)


Epoch [1/10] Loss: 0.0055
Epoch [2/10] Loss: 0.0350
Epoch [3/10] Loss: 0.0034
Epoch [4/10] Loss: 0.2987
Epoch [5/10] Loss: 1.3541
Epoch [6/10] Loss: 0.0019
Epoch [7/10] Loss: 0.0008


In [10]:
print(total)

400


In [11]:
torch.save(model, "trainedModel")